In [39]:
#packages
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [ ]:
'''
start and end date format'%Y-%m-%d'
valid periods ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
'''
class Yahoo_scraper:
    def __init__(self, ticker_name: str = 'EURINR=X', start_date: str = None, end_date: str = None, interval: str ='1d'):
        self.ticker_name = ticker_name
        self.start_date =start_date
        self.end_date = end_date
        self.interval = interval
        fig1 = go.Figure()
        fig2 = go.Figure()
        self.fig1 = fig1
        self.fig2 = fig2
        # Buy and Sell signals 
        self.buy_signal = []  
        self.sell_signal = []
    def historical_data(self,start_date:str = None,end_date:str = None,interval:str = None)->pd.DataFrame:
        stock = yf.Ticker(self.ticker_name)
        data = stock.history(start=start_date if start_date else self.start_date,
                             end =end_date if end_date else self.end_date,
                             interval = interval if interval else self.interval)
        required_data = data.loc[:,['Open','High','Low','Close']]
        return required_data
    # Moving Average
    def indicator_moving_average(self,window:int = 9,offset:int =0)->pd.DataFrame:
        new_start_date=str((pd.to_datetime(self.start_date)-pd.Timedelta(days=window+offset+3)).date())
        data = self.historical_data(start_date = new_start_date)
        data['moving_average'] = data['Close'].rolling(window=window).mean()

        return data.dropna()
    #  Bollinger Band
    def indicator_bollinger_band(self,window:int = 9)-> pd.DataFrame:
        data = self.indicator_moving_average(window,window+3)
        # Calculate the rolling standard deviation with a window size of 3
        data['std_dev'] = data['Close'].rolling(window).std()
        # Calculate the upper Bollinger Band
        data['upper_band'] = data['moving_average'] + (data['std_dev'] * 2)
        # Calculate the lower Bollinger Band
        data['lower_band'] = data['moving_average'] - (data['std_dev'] * 2)
        data= data.drop(columns=['moving_average'])
        return data.dropna()
    
    # CCI (Commodity Channel Index)
    def indicator_CCI(self,window:int=9)-> pd.DataFrame:
        new_start_date=str((pd.to_datetime(self.start_date)-pd.Timedelta(days=window+3)).date())
        data = self.historical_data(start_date = new_start_date)
        # Calculate the Typical Price (TP)
        data['TP'] = (data['High'] + data['Low'] + data['Close']) / 3
        # Calculate the Moving Average of the Typical Price (SMA_TP) with a window size of 3
        data['SMA_TP'] = data['TP'].rolling(window=window).mean()
        # Define a custom function to calculate Mean Absolute Deviation
        def mad(series):
            return (series - series.mean()).abs().mean()
        # Calculate the Mean Deviation
        data['Mean_Deviation'] = data['TP'].rolling(window=window).apply(lambda x: mad(pd.Series(x)),raw =False)
        # Calculate the CCI
        data['CCI'] = (data['TP'] - data['SMA_TP']) / (0.015 * data['Mean_Deviation'])
        return data.dropna()
    
    #plotting
    def candle_stick_chart(self,moving_average:bool =False,
                           bolliger_band:bool = False,
                           CCI :bool =False,
                           signal:bool =False,
                           bollinger_band_window:int = 9,
                           moving_average_window:int = 9,
                           CII_window:int = 9):
        data = self.indicator_bollinger_band(bollinger_band_window)
        data['moving_average'] = self.indicator_moving_average(moving_average_window)['moving_average']
        data['CCI'] = self.indicator_CCI(CII_window)['CCI']
        self.fig1.add_trace(go.Candlestick(x=data.index,
                    open=data['Open'],  # Placeholder for open price
                    high=data['High'],  # Placeholder for high price
                    low=data['Low'],    # Placeholder for low price
                    close=data['Close'],
                    name=self.ticker_name))
        if moving_average:
            self.fig1.add_trace(go.Scatter(x=data.index, y=data['moving_average'],
                            mode='lines', name='Moving Average', line=dict(color='black', dash='dash')))
        if bolliger_band:
            self.fig1.add_trace(go.Scatter(x=data.index, y=data['upper_band'],
                         mode='lines', name='Upper Bollinger Band', line=dict(color='orange', dash='dash')))
            self.fig1.add_trace(go.Scatter(x=data.index, y=data['lower_band'],
                         mode='lines', name='Lower Bollinger Band', line=dict(color='orange', dash='dash')))
            # Add fill between for Bollinger Bands
            self.fig1.add_trace(go.Scatter(x=data.index, y=data['upper_band'],
                                    mode='lines', line=dict(color='gray', width=0),
                                    fill='tonexty', fillcolor='rgba(211, 211, 211, 0.3)', name='Bollinger Bands Fill'))
        if CCI:
            # Add CCI plot
            self.fig2.add_trace(go.Scatter(x=data.index, y=data['CCI'],
                                    mode='lines', name='CCI', line=dict(color='orange')))

            # Add horizontal lines for CCI overbought/oversold
            self.fig2.add_hline(y=100, line=dict(color='red', dash='dash'), name='Overbought')
            self.fig2.add_hline(y=-100, line=dict(color='red', dash='dash'), name='Oversold')

            # Update layout for the second figure
            self.fig2.update_layout(title='Commodity Channel Index (CCI)',
                            xaxis_title='Date',
                            yaxis_title='CCI',
                            xaxis_rangeslider_visible=False)
        
        if signal:  
            # Add Buy arrows
            self.fig1.add_trace(go.Scatter(x=data.index[self.buy_signal], y=data['Low'][self.buy_signal] - 1,  # Slightly below the candlestick low
                                    mode='markers+text', 
                                    marker=dict(symbol='triangle-up', color='green', size=15), 
                                    text=['Buy']*len(self.buy_signal), textposition="bottom center",
                                    name='Buy Signal'))

            # Add Sell arrows
            self.fig1.add_trace(go.Scatter(x=data.index[self.sell_signal], y=data['High'][self.sell_signal] + 1,  # Slightly above the candlestick high
                                    mode='markers+text', 
                                    marker=dict(symbol='triangle-down', color='red', size=15), 
                                    text=['Sell']*len(self.sell_signal), textposition="top center",
                                    name='Sell Signal'))
            
        # Update layout for the first figure1
        self.fig1.update_layout(title='Moving Average',
                        xaxis_title='Date',
                        yaxis_title='Price',
                        xaxis_rangeslider_visible=False)
        # Show the second figure
        self.fig1.show()
        if CCI:
            self.fig2.show()

In [168]:
test_object = Yahoo_scraper(start_date = '2024-01-18',end_date = None,interval = '1d')
test_object.candle_stick_chart(moving_average =True,
                           bolliger_band= False,
                           CCI  =False,
                           signal=True,
                           moving_average_window= 50,
                           bollinger_band_window= 9,
                           CII_window = 20)
# test_object.historical_data()['Low']